In [2]:
import os.path as op
import pandas as pd
import glob
import numpy as np

In [3]:
def find_datafiles(filename):
    f = glob.glob(filename)
    return(f)

In [4]:
data_folder = "C:\\Users\\LocalAdmin\\Documents\\GUESS\\Guess_fMRI\\data"

In [5]:
# find renamed data files to identify relevant log files and obtain ID
filename = op.join(data_folder, 'pilot2_*.csv')
file_list = find_datafiles(filename)
frames = []
print(filename)
print(file_list)
# loop through file list
for f in file_list:
    split_fname = f.split('_') # split name into chunks seperated by '_'
    pid = split_fname[1].split('\\')[-1] # second chunk is the ID
    date = split_fname[-2] # second to last bit is date
    time = split_fname[-1].split('.') # last bit is time + extension
    time.pop()
    time = '.'.join(time)
    
    # find log file matching date and time
    log_fname = pid + "_guess_" + date + '_' + time + ".log" # pilot2_guess_2025-04-22_13h19.30.422.log
    log_fpath = op.join(data_folder, log_fname)
    log_file_list = find_datafiles(log_fpath)
    print(log_file_list)
    if len(log_file_list) > 1:
        print('Warning! More than 1 log file.')
    elif len(log_file_list) < 1:
        print('Warning: No log file attached to data sheet.')
        print('\t affected data: ', date, time, id)
        continue
    log_file = log_file_list[0] # open the first occurence
    log_file = open(log_file, "r")
    log = log_file.read()
    log_part1_part2 = ''

    #Learning
    if "end_part2_text: autoDraw = True" in log: # check if participant finished the task
        log_part1_part2 = log.split('end_part1_text: autoDraw = True')[0] #split learning and recall
        if 'start_task_text: autoDraw = True' in log_part1_part2: 
            log_part1_part2 = log_part1_part2.split('start_task_text: autoDraw = True')
            log_part1_part2 = log_part1_part2[1] # we don't care about practice
        trials = log_part1_part2.split("cue_stim_learning: text = ")
        trials.pop(0) # first element is irrelevant, because it precedes the occurence of the word
        print(len(trials)) # sanity check: should be 100 if they finished
        indeces = []
        cues = []
        i = 1
        for trial in trials:
            lines = trial.split('\n')
            cue = lines[0].split("\'")[1] # first line contains the cue in ''
            cues.append(cue)
            indeces.append(i)
            i += 1
        part1 = ["Learning"] * 80

        #Recall
        log_part1_part2 = log.split('end_part1_text: autoDraw = True')[1] #split learning and recall
        if 'start_task_text: autoDraw = True' in log_part1_part2: 
            log_part1_part2 = log_part1_part2.split('start_task_text: autoDraw = True')
            log_part1_part2 = log_part1_part2[1] # we don't care about practice
        trials = log_part1_part2.split("cue_stim_recall: text = ")
        trials.pop(0) # first element is irrelevant, because it precedes the occurence of the word
        print(len(trials)) # sanity check: should be 100 if they finished
        
        i = 1
        for trial in trials:
            lines = trial.split('\n')
            cue = lines[0].split("\'")[1] # first line contains the cue
            cues.append(cue)
            indeces.append(i)
            i += 1

        odata = pd.read_csv(f)
        odata.loc[odata["test_trials.thisRepN"] >= 0, "Cue"] = cues[80:]
        odata.to_csv(f)
        
        part2 =  ["Recall"] * 80
        part = part1 + part2
        data = np.array([indeces, part, cues]).transpose()
        column_values = ['Trial', 'Phase', 'Cue',]
        # creating the dataframe
        df = pd.DataFrame(data = data,  
                          columns = column_values)
        
        df.to_csv(op.join(data_folder, pid + '_cues.csv'))

C:\Users\LocalAdmin\Documents\GUESS\Guess_fMRI\data\pilot2_*.csv
['C:\\Users\\LocalAdmin\\Documents\\GUESS\\Guess_fMRI\\data\\pilot2_cues.csv', 'C:\\Users\\LocalAdmin\\Documents\\GUESS\\Guess_fMRI\\data\\pilot2_guess_2025-04-22_13h19.30.422.csv', 'C:\\Users\\LocalAdmin\\Documents\\GUESS\\Guess_fMRI\\data\\pilot2_guess_2025-04-22_14h10.33.811.csv', 'C:\\Users\\LocalAdmin\\Documents\\GUESS\\Guess_fMRI\\data\\pilot2_guess_2025-04-22_14h15.26.228.csv', 'C:\\Users\\LocalAdmin\\Documents\\GUESS\\Guess_fMRI\\data\\pilot2_part0_2025-04-22_13h07.58.863.csv']
[]
	 affected data:  fMRI\data\pilot2 cues <built-in function id>
['C:\\Users\\LocalAdmin\\Documents\\GUESS\\Guess_fMRI\\data\\pilot2_guess_2025-04-22_13h19.30.422.log']
80
80
['C:\\Users\\LocalAdmin\\Documents\\GUESS\\Guess_fMRI\\data\\pilot2_guess_2025-04-22_14h10.33.811.log']
['C:\\Users\\LocalAdmin\\Documents\\GUESS\\Guess_fMRI\\data\\pilot2_guess_2025-04-22_14h15.26.228.log']
[]
	 affected data:  2025-04-22 13h07.58.863 <built-in funct

In [6]:
odata.loc[odata["test_trials.thisRepN"] >= 0, "Cue"]

99          Gast
100      Shampoo
101         Floh
102        Brise
103       Safari
         ...    
174    Scheidung
175       Vampir
176         Zimt
177       Winkel
178        Kamin
Name: Cue, Length: 80, dtype: object

In [7]:
df

,Trial,Phase,Cue
0,1,Learning,Geschwister
1,2,Learning,Helm
2,3,Learning,Zug
3,4,Learning,Vampir
4,5,Learning,Nudeln
...,...,...,...
155,76,Recall,Scheidung
156,77,Recall,Vampir
157,78,Recall,Zimt
158,79,Recall,Winkel
